## Fixing Your Mistakes by Updating and Overwriting Data

### Setup the Environment

In [1]:
import os
import sys
import tempfile
import pandas as pd
import arcgis
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection

### Connect to the GIS

In [2]:
gis = GIS(profile='your_online_profile')

## Load the Sample Data

- in this tutorial the data used will be downloaded from the ArcGIS Open Data Site


In [3]:
df = pd.read_csv("https://opendata.arcgis.com/datasets/dc3289eab3d2400ea49c154863312434_8.csv")
df.head()

,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,BLOCK_GROUP,CENSUS_TRACT,VOTING_PRECINCT,LATITUDE,LONGITUDE,BID,START_DATE,END_DATE,OBJECTID,OCTO_RECORD_ID
0,-77.043471,38.897846,23006624,2023/01/13 05:25:22+00,MIDNIGHT,OTHERS,THEFT/OTHER,600 - 699 BLOCK OF 19TH STREET NW,396229.23,136666.18,...,010800 6,10800,Precinct 2,38.897838,-77.043469,NaN,2023/01/13 04:20:00+00,2023/01/14 04:30:00+00,284143890,NaN
1,-77.001312,38.889816,23006626,2023/01/13 10:19:27+00,MIDNIGHT,OTHERS,THEFT/OTHER,300 - 399 BLOCK OF EAST CAPITOL STREET,399886.38,135773.92,...,008200 2,8200,Precinct 89,38.889808,-77.001310,CAPITOL HILL,2023/01/13 04:36:00+00,2023/01/13 05:54:00+00,284143891,NaN
2,-77.021808,38.967941,23006627,2023/01/13 05:27:24+00,MIDNIGHT,OTHERS,THEFT/OTHER,600 - 699 BLOCK OF TEWKESBURY PLACE NW,398110.33,144446.71,...,001901 4,1901,Precinct 59,38.967933,-77.021805,NaN,2023/01/11 20:00:00+00,2023/01/14 04:00:00+00,284143892,NaN
3,-76.993083,38.924518,23006631,2023/01/13 05:31:50+00,MIDNIGHT,OTHERS,MOTOR VEHICLE THEFT,800 - 999 BLOCK OF EVARTS STREET NE,400600.00,139626.19,...,009302 1,9302,Precinct 74,38.924510,-76.993081,NaN,2023/01/13 05:13:00+00,2023/01/13 05:31:00+00,284143893,NaN
4,-77.004495,38.928807,23006636,2023/01/13 06:46:16+00,MIDNIGHT,GUN,ROBBERY,100 - 379 BLOCK OF MICHIGAN AVENUE NE,399610.40,140102.24,...,002302 2,2302,Precinct 44,38.928799,-77.004493,NaN,2023/01/13 05:01:00+00,2023/01/13 05:02:00+00,284143894,NaN


In [4]:
split_spot = len(df)//2
half_df = df[:split_spot].copy()

## Add the Sample Data to GIS 

In [5]:
half_df.to_csv("./myuploaddata.csv")

In [6]:
item = gis.content.add({
    'title' : 'overwrite_source',
    'type' : 'CSV',
    'tags' : "demo, data"
},
data="./myuploaddata.csv")
item

<Item title:"overwrite_source" type:CSV owner:andrew57>

In [7]:
crime_locations = item.publish()
crime_locations

<Item title:"overwrite_source" type:Feature Layer Collection owner:andrew57>

### OH NO WE FORGOT HALF THE DATA

<img src="./img/shame.jpg"/>

- how do we fix this?

### Enter Overwriting Data

- new data can be uploaded to the service and republished.  
- the `overwrite` parameter needs to be set to `True`
- the csv file needs to the same exact file name and same column names 
    + if you do not have this, you will fail 

In [8]:
import os
with tempfile.TemporaryDirectory() as tmp:
    fp = os.path.join(tmp, "myuploaddata.csv") # same name CHECK
    df.to_csv(fp)
    flc = FeatureLayerCollection.fromitem(crime_locations)
    flc.manager.overwrite(fp)

**What Happened Here?**

1. Write the data to a temporary folder
2. Access the item's `Feature Service`
3. Access the manager (owners and admins only)
4. Call `overwrite` and pass in the new dataset

In [10]:
new_count = crime_locations.layers[0].query(return_count_only=True) 
old_count = len(half_df)
print(new_count, old_count, new_count > old_count)

2534 1267 True


In [11]:
assert crime_locations.delete()
assert item.delete()

**Shame No More!**